In [1]:
import time
import sys
import numpy as np
import tensorflow as tf
sys.path.append("..")
import udc_model
import udc_hparams
from models.dual_encoder import dual_encoder_model
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from termcolor import colored

In [2]:
# define flags
tf.flags.DEFINE_string("model_dir", "../runs/1507611689Stage2/", "Directory to load model checkpoints from")
tf.flags.DEFINE_string("vocab_processor_file", "../data/vocab_processor.bin", "Saved vocabulary processor file")
FLAGS = tf.flags.FLAGS

In [3]:
# define functions
def tokenizer_fn(iterator):
  return (x.split(" ") for x in iterator)

def get_features(context, utterances):
  context_matrix = np.array(list(vp.transform([context])))
  utterance_matrix = np.array(list(vp.transform([utterances])))
  context_len = len(context.split(" "))
  utterance_len = len(utterances.split(" "))
  features =  {
        "context": tf.convert_to_tensor(context_matrix, dtype=tf.int64),
        "context_len": tf.constant(context_len, shape=[1,1], dtype=tf.int64),
        "utterance": tf.convert_to_tensor(utterance_matrix, dtype=tf.int64),
        "utterance_len": tf.constant(utterance_len, shape=[1,1], dtype=tf.int64),
        "len":len(utterances)
  }

  return features, None

def findCandidates(hostnport, database, context, histIDs):
    url = 'http://'+hostnport+'/action=query&MaxResults=10&PrintFields=title&text='+context+'&DatabaseMatch=' + database + '&DontMatchID=' + histIDs
    
    response = requests.get(url)

    # parse xml style strings
    tree = ET.ElementTree(ET.fromstring(response.text))
    root = tree.getroot()

    # Get utterances
    ns = {'autn': 'http://schemas.autonomy.com/aci/'}

    utterances = []
    responsedata = root.find('responsedata')
    #print (len(responsedata.findall('autn:hit', ns)))
    if len(responsedata.findall('autn:hit', ns)) == 0:
      print ("no response")
      return None
    for hit in responsedata.findall('autn:hit', ns):
      #reference = hit.find('autn:reference', ns). #reference = hit.find('autn:reference', ns).text
      DocId = hit.find('autn:id', ns).text
      weight = hit.find('autn:weight', ns).text
      utterance =  hit.find('autn:title', ns).text
      utteranceRow = [DocId, weight, utterance]
      utterances.append(utteranceRow)

    utterancesDF = pd.DataFrame(utterances)
    utterancesDF.columns = ['DocId', 'weight', 'utterance']
    
    print("\nPotential responses from chat history:")
    with pd.option_context('display.max_colwidth', 3000):
        display(utterancesDF)
    return utterancesDF

In [4]:
# Load vocabulary
vp = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(
  FLAGS.vocab_processor_file)

In [5]:
def rankResponses(input_context, responses_df):
    
    POTENTIAL_RESPONSES = responses_df['utterance']
    WEIGHTS = responses_df['weight']
    DOCIDS = responses_df['DocId']
        
    answerId=0
    MaxProb=0
    utterances = []
    
    for i in range(0,len(POTENTIAL_RESPONSES)):
        r = POTENTIAL_RESPONSES[i];
    
        prob = estimator.predict(input_fn=lambda: get_features(input_context, r))
        currentProb = next(prob)[0]
        
        utteranceRow = [i, DOCIDS[i], currentProb, r]
        utterances.append(utteranceRow)
        
    utterancesDF = pd.DataFrame(utterances)
    utterancesDF.columns = ['AnswerId', 'DocId', 'Probability', 'utterance']
    
    sort_by_probDF = utterancesDF.sort_values('Probability',ascending=False) 
    
    print("\nProbability of potential responses:")
    with pd.option_context('display.max_colwidth', 3000):
        display(sort_by_probDF)
    print('\n')
    answerId = sort_by_probDF.iloc[0]['AnswerId']
   
    return answerId

In [ ]:
if __name__ == "__main__":
    
  ''' _level_names = {
    FATAL: 'FATAL',
    ERROR: 'ERROR',
    WARN: 'WARN',
    INFO: 'INFO',
    DEBUG: 'DEBUG',
  }
  '''
  #print('Welcome to DeepVirtualAgent v0.1 !')
  #print()
  #print('TensorFlow detected: v{}'.format(tf.__version__))

  tf.logging.set_verbosity(tf.logging.WARN)
  hparams = udc_hparams.create_hparams()
  model_fn = udc_model.create_model_fn(hparams, model_impl=dual_encoder_model)

  estimator = tf.contrib.learn.Estimator(model_fn=model_fn, model_dir=FLAGS.model_dir)

  hostnport = '10.30.30.205:9000'
  database = 'Dialog'
  context ='' 
  histIDs = ''

  utteranceTag1 = ' __eou__ __eot__ '
  utteranceTag2 = ' __eot__ '
    
  while 1:
    question = input("User input: ")
    
    if question.lower() in ['quit','bye','good bye']: 
        break
    context += question
    context += utteranceTag1
    
    INPUT_CONTEXT = context
    
    POTENTIAL_RESPONSES_DF = findCandidates(hostnport, database, context, histIDs)
    
    histIDs = histIDs.split('+')[-1]

    
    if (POTENTIAL_RESPONSES_DF is not None):
        answerId = rankResponses(INPUT_CONTEXT, POTENTIAL_RESPONSES_DF)

        POTENTIAL_RESPONSES = POTENTIAL_RESPONSES_DF['utterance']
        WEIGHTS = POTENTIAL_RESPONSES_DF['weight']
        DOCIDS = POTENTIAL_RESPONSES_DF['DocId']
        '''
        starttime = time.time()
        answerId=0
        MaxProb=0
        print("\nProbability of potential responses:")
        for i in range(0,len(POTENTIAL_RESPONSES)):
          r = POTENTIAL_RESPONSES[i];
    
          prob = estimator.predict(input_fn=lambda: get_features(INPUT_CONTEXT, r))
        # fixed based on issue comment in githubs
        # https://github.com/dennybritz/chatbot-retrieval/issues/60
        #print("{}: {:g}".format(r, prob[0,0]))
          currentProb = next(prob)[0]
          #Apply weight to the prob
          #currentProb = currentProb * float(WEIGHTS[i]) * 0.01
  
        #print("{}: {:g}".format(i, next(prob)[0])) 
          #print("Potential response: {}".format(r))
          print("{}: {:g}: {}".format(i, currentProb,r))
        
          if currentProb >= MaxProb:
              answerId = i
              MaxProb = currentProb
    
          endtime = time.time()

        print('\n')
        ''''''print('[Predict time]',"%.2f sec" % round(endtime - starttime,2))
        print('[     Context]',INPUT_CONTEXT)
        print("[Best answer has probability of {} at {} ]".format(MaxProb, answerId))
        '''
        #print('Virtual Agent: ', POTENTIAL_RESPONSES[answerId])
        print(colored('[Virtual Agent: ]', on_color='on_green',color="white"), POTENTIAL_RESPONSES[answerId])
        context += POTENTIAL_RESPONSES[answerId]
        context += utteranceTag2
        histIDs += '+'+DOCIDS[answerId]
        
    
print ('\nGood Bye!')

User input: I can’t connect my mailbox with my mobile phone

Potential responses from chat history:


,DocId,weight,utterance
0,499880,89.00,Go here -> here -> here and select this __eou__
1,499879,88.99,Please go here -> here -> here and let us know which ios version you run. __eou__
2,499902,88.99,Please go here -> here -> here and let us know which ios version you run. __eou__
3,499901,88.94,Which error message do you receive ? __eou__
4,499878,88.94,Which error message do you receive ? __eou__
5,499900,88.94,Which mobile phone are you using? __eou__
6,499877,88.94,Which mobile phone are you using? __eou__
7,264359,69.06,yes .. but whi .. it work on the livecd __eou__
8,235306,69.05,be that an sd card or similar that 's slot into the mobil phone ? __eou__
9,307737,69.02,yeah __eou__



Probability of potential responses:


,AnswerId,DocId,Probability,utterance
5,5,499900,0.999995,Which mobile phone are you using? __eou__
6,6,499877,0.999995,Which mobile phone are you using? __eou__
1,1,499879,0.999986,Please go here -> here -> here and let us know which ios version you run. __eou__
2,2,499902,0.999986,Please go here -> here -> here and let us know which ios version you run. __eou__
0,0,499880,0.999911,Go here -> here -> here and select this __eou__
3,3,499901,0.998531,Which error message do you receive ? __eou__
4,4,499878,0.998531,Which error message do you receive ? __eou__
9,9,307737,0.550445,yeah __eou__
7,7,264359,0.211671,yes .. but whi .. it work on the livecd __eou__
8,8,235306,0.159030,be that an sd card or similar that 's slot into the mobil phone ? __eou__




[Virtual Agent: ] Which mobile phone are you using? __eou__
User input: iPhone 6

Potential responses from chat history:


...



[Virtual Agent: ] Please go here -> here -> here and let us know which ios version you run. __eou__
User input: 10.4

Potential responses from chat history:


,DocId,weight,utterance
0,499880,85.16,Go here -> here -> here and select this __eou__
1,499878,70.03,Which error message do you receive ? __eou__
2,499877,62.49,Which mobile phone are you using? __eou__
3,304521,59.57,"look at winff , ffmpeg can grab video commandlin apprent , winff be in the repo as a frontend but it dosent seem to make use of the webcam __eou__ for the next guy that ask , http : //guvcview.berlios.de/index.html be the captur solut __eou__"
4,362960,59.48,i see . well i see it when i run iwconfig ' command but i do n't see anyth under ifconfig ' . i do n't know what i must do to get that option ? __eou__ be you run ubuntu 10.04 lts ? __eou__
5,277401,58.50,hmm . mayb you have configur somewher that your pc should deni incom file . but basic if the devic be pari it should be possibl to send file easili in both way . it work here also with window mobil 6 . __eou__
6,395417,58.34,modifi auto eth0 in ach system from automat dhcp to manual __eou__ dedd ? __eou__
7,25510,58.20,"that 's ok , i have it in my phone __eou__ but when i select a pictur from the phone and tri to send it via bluetooth , it search for nearbi bluetooth devic and doe n't find anyith __eou__"
8,431198,58.17,have you creat an entri for it in network manag under mobil broadband ? __eou__
9,85691,58.12,"then i dont know , what be you use mobil phone to connect ? __eou__"



Probability of potential responses:


,AnswerId,DocId,Probability,utterance
2,2,499877,0.999991,Which mobile phone are you using? __eou__
0,0,499880,0.999983,Go here -> here -> here and select this __eou__
1,1,499878,0.999847,Which error message do you receive ? __eou__
7,7,25510,0.979073,"that 's ok , i have it in my phone __eou__ but when i select a pictur from the phone and tri to send it via bluetooth , it search for nearbi bluetooth devic and doe n't find anyith __eou__"
6,6,395417,0.574756,modifi auto eth0 in ach system from automat dhcp to manual __eou__ dedd ? __eou__
9,9,85691,0.268670,"then i dont know , what be you use mobil phone to connect ? __eou__"
8,8,431198,0.104540,have you creat an entri for it in network manag under mobil broadband ? __eou__
3,3,304521,0.036245,"look at winff , ffmpeg can grab video commandlin apprent , winff be in the repo as a frontend but it dosent seem to make use of the webcam __eou__ for the next guy that ask , http : //guvcview.berlios.de/index.html be the captur solut __eou__"
4,4,362960,0.000131,i see . well i see it when i run iwconfig ' command but i do n't see anyth under ifconfig ' . i do n't know what i must do to get that option ? __eou__ be you run ubuntu 10.04 lts ? __eou__
5,5,277401,0.000113,hmm . mayb you have configur somewher that your pc should deni incom file . but basic if the devic be pari it should be possibl to send file easili in both way . it work here also with window mobil 6 . __eou__




[Virtual Agent: ] Which mobile phone are you using? __eou__
